# Laboratorio #4 - Familias de Malware

# Parte 1

In [341]:
import os
import pefile
import pandas as pd
import hashlib
import re
from datetime import datetime, timezone
import subprocess

## Creación del dataset

In [342]:
MALWARE_DIR = "./MALWR"
malware_files = [os.path.join(MALWARE_DIR, f) for f in os.listdir(MALWARE_DIR) if os.path.isfile(os.path.join(MALWARE_DIR, f))]

print(f"🔍 Se encontraron {len(malware_files)} archivos de malware en el directorio.")

🔍 Se encontraron 40 archivos de malware en el directorio.


In [343]:
def convert_timestamp(timestamp):
    try:
        return datetime.fromtimestamp(timestamp, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
    except (OSError, OverflowError, ValueError):
        return "Invalid Timestamp"

# 📌 Función para calcular el SHA-256 del archivo
def get_sha256(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

# 📌 Función para detectar empaquetado (UPX u otros)
def detect_packed(pe):
    packed_sections = [b"UPX0", b"UPX1", b"UPX2", b"MPRESS1", b"MPRESS2"]
    suspicious_flags = 0

    for section in pe.sections:
        section_name = section.Name.rstrip(b'\x00')

        # 📌 Detectar nombres de secciones típicas de packers (UPX, MPRESS, etc.)
        if section_name in packed_sections:
            return 1  # UPX u otro packer detectado

        # 📌 Detectar flags sospechosos (ejecutable + escritura activados)
        characteristics = section.Characteristics
        if (characteristics & 0x20000000) and (characteristics & 0x80000000):
            suspicious_flags += 1

    # 📌 Si varias secciones tienen flags sospechosas, es posible que esté empaquetado
    if suspicious_flags > 1:
        return 1

    # 📌 Detectar si el PE no tiene importaciones (muy sospechoso)
    if not hasattr(pe, "DIRECTORY_ENTRY_IMPORT") or len(pe.DIRECTORY_ENTRY_IMPORT) == 0:
        return 1  # Probablemente empaquetado

    return 0  # No parece estar empaquetado

# 📌 Función para obtener funciones importadas y APIs sospechosas
suspicious_api_calls = {
    "process hollowing": ["CreateProcess", "GetProcAddress", "VirtualAllocEx", "WriteProcessMemory"],
    "create remote thread": ["OpenProcess", "CreateRemoteThread", "WriteProcessMemory"],
    "networking": ["WSAStartup", "socket", "bind", "connect", "recv"],
    "code injection": ["VirtualProtect", "VirtualAlloc", "LoadLibrary"]
}

def extract_sections(pe):
    section_data = []
    for section in pe.sections:
        section_name = section.Name.rstrip(b'\x00').decode(errors="ignore")
        virtual_address = hex(section.VirtualAddress)
        virtual_size = hex(section.Misc_VirtualSize)
        raw_size = section.SizeOfRawData
        characteristics = hex(section.Characteristics)

        section_data.append({
            "Name": section_name,
            "VirtualAddress": virtual_address,
            "VirtualSize": virtual_size,
            "RawSize": raw_size,
            "Characteristics": characteristics
        })

    return section_data

def extract_imports(pe):
    imported_functions = []
    imported_dlls = []
    detected_suspicious_apis = set()

    if hasattr(pe, "DIRECTORY_ENTRY_IMPORT"):
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            dll_name = entry.dll.decode(errors="ignore")
            imported_dlls.append(dll_name)

            for function in entry.imports:
                func_name = function.name.decode(errors="ignore") if function.name else "N/A"
                imported_functions.append(func_name)

                # Detectar API sospechosas
                for category, apis in suspicious_api_calls.items():
                    if func_name in apis:
                        detected_suspicious_apis.add(func_name)

    return {
        "DLLs": imported_dlls,
        "Functions": imported_functions,
        "SuspiciousAPIs": list(detected_suspicious_apis)
    }
    

def get_imported_functions(pe):
    imported_functions = []
    detected_suspicious_apis = set()

    if hasattr(pe, "DIRECTORY_ENTRY_IMPORT"):
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            for function in entry.imports:
                func_name = function.name.decode("utf-8") if function.name else "N/A"
                imported_functions.append(func_name)

                # Detectar APIs sospechosas
                for category, apis in suspicious_api_calls.items():
                    if func_name in apis:
                        detected_suspicious_apis.add(func_name)
    
    # 📌 Validar si hay importaciones después del desempaquetado
    if not imported_functions:
        print("⚠️ No se encontraron importaciones tras desempaquetar. Posible empaquetado avanzado.")
    
    return ", ".join(imported_functions), ", ".join(detected_suspicious_apis) if detected_suspicious_apis else "None"

# 📌 Función para detectar secciones sospechosas
def detect_suspicious_sections(pe):
    for section in pe.sections:
        name = section.Name.decode().strip()
        size = section.SizeOfRawData
        
        if name == ".text" and size < 1024:
            return 1  # Small .text (Possible Packing)
        if name == ".rsrc" and size > 1000000:
            return 2  # Large .rsrc (Possible Malware)
    return 0  # Normal

# 📌 Función para extraer strings sospechosas (URLs, IPs, comandos)
def extract_strings(file_path):
    with open(file_path, "rb") as f:
        data = f.read()
    
    strings = re.findall(b"[ -~]{4,}", data)
    decoded_strings = [s.decode("utf-8", "ignore") for s in strings]

    suspicious_keywords = ["http", "ftp", "cmd", "powershell", "wget", "curl", "0.0.0.0"]
    suspicious_count = sum(1 for s in decoded_strings if any(k in s for k in suspicious_keywords))
    
    return suspicious_count  # Returns count instead of full strings

In [344]:
def unpack_upx(file_path):
    """ Intenta desempaquetar un archivo UPX si está empaquetado """
    try:
        print(f"🔄 Intentando desempaquetar {file_path} con UPX...")
        result = subprocess.run(["upx", "-d", file_path], capture_output=True, text=True)

        # 📌 Validar si el desempaquetado realmente ocurrió
        if "Unpacked" in result.stdout or "unpacked" in result.stdout:
            print(f"✅ Archivo {file_path} desempaquetado exitosamente.")
            return True
        else:
            print(f"⚠️ UPX no pudo desempaquetar {file_path}. Posible otro packer.")
            return False
    except Exception as e:
        print(f"⚠️ No se pudo desempaquetar {file_path}. Error: {e}")
        return False

def extract_pe_info(file_path):
    try:
        pe = pefile.PE(file_path)

        # 📌 Si el archivo está empaquetado, intentar desempaquetarlo con UPX
        if detect_packed(pe):
            print(f"⚠️ Archivo {file_path} parece estar empaquetado.")
            if unpack_upx(file_path):
                print(f"🔄 Reanalizando {file_path} después del desempaquetado...")
                pe = pefile.PE(file_path)  # Re-analizar el PE desempaquetado
        
        # 📌 Extraer importaciones
        imports_data = extract_imports(pe)

        # 📌 Extraer información general
        pe_info = {
            "Filename": os.path.basename(file_path),
            "SHA256": get_sha256(file_path),
            "TimeDateStamp": convert_timestamp(pe.FILE_HEADER.TimeDateStamp),
            "Subsystem": pe.OPTIONAL_HEADER.Subsystem,
            "DLLCharacteristics": pe.OPTIONAL_HEADER.DllCharacteristics,
            "NumberOfSections": pe.FILE_HEADER.NumberOfSections,
            "Packed": detect_packed(pe),  # 0 = No, 1 = Sí
            "Suspicious_Sections": detect_suspicious_sections(pe),
            "Num_DLLs": len(imports_data["DLLs"]),
            "Num_Imports": len(imports_data["Functions"]),
            "Num_Suspicious_APIs": len(imports_data["SuspiciousAPIs"]),
            "Strings": extract_strings(file_path),
        }

        pe.close()
        return pe_info
    
    except Exception as e:
        print(f"⚠️ Error analizando {file_path}: {e}")
        return None

In [345]:
dataset = []
for file in malware_files:
    if not os.path.isfile(file):  # Ignorar archivos inexistentes
        continue
    pe_data = extract_pe_info(file)
    if pe_data:
        dataset.append(pe_data)

df = pd.DataFrame(dataset)

# 📌 Eliminar columnas si están completamente vacías o duplicadas
columns_to_drop = []
if df["Suspicious_Sections"].nunique() == 1:
    columns_to_drop.append("Suspicious_Sections")

if df["Packed"].nunique() == 1:
    columns_to_drop.append("Packed")

if "DLLCharacteristics" in df.columns and df["DLLCharacteristics"].nunique() == 1:
    columns_to_drop.append("DLLCharacteristics")

# 📌 Eliminar columnas innecesarias
df.drop(columns=columns_to_drop, inplace=True)

# 📌 Guardar dataset limpio
df.to_csv("malware_dataset_optimized.csv", index=False)
print("✅ Dataset optimizado guardado como 'malware_dataset_optimized.csv'")

# 📌 Verificar datos finales
display(df.head())

# 📌 Guardar el dataset
df.to_csv("malware_dataset_cleaned.csv", index=False)
print("✅ Dataset guardado como 'malware_dataset_cleaned.csv'")

✅ Dataset optimizado guardado como 'malware_dataset_optimized.csv'


,Filename,SHA256,TimeDateStamp,Subsystem,NumberOfSections,Num_DLLs,Num_Imports,Num_Suspicious_APIs,Strings
0,JH78C0A33A1B472A8C16123FD696A5CE5EBB,14af57c8a6a37b2fa9bd738285b2951b63fcfb34e1450c...,2009-05-14 17:12:40,2,4,5,62,4,4
1,NBV_8B75BCBFF174C25A0161F30758509A44,f52af02529b12c20e45618cae1a38c4ace06f5496ce160...,2009-05-14 17:12:40,2,4,5,62,4,4
2,POL55_A4F1ECC4D25B33395196B5D51A06790,57ef37c30a4a10b334a745d2f6235b4703ae99cf8e64e1...,2011-07-29 07:10:31,2,3,7,85,1,5
3,6FAA4740F99408D4D2DDDD0B09BBDEFD,a5244829aa94b009e479f06f7ae3e9f40daef32b1570c4...,2009-05-14 17:12:40,2,4,5,62,4,4
4,A316D5AECA269CA865077E7FFF356E7D,02e0d550c86910a19ab73ec02bcbe4ed095d683414db2f...,2009-05-14 17:12:40,2,4,5,62,4,4


✅ Dataset guardado como 'malware_dataset_cleaned.csv'


## Exploración y pre-procesamiento de datos

In [346]:
df = pd.read_csv("malware_dataset_cleaned.csv")

In [347]:
# Mostrar las primeras filas
display(df.head())

# Revisar los tipos de datos
print("\n📌 Tipos de datos en cada columna:")
print(df.dtypes)

# Revisar valores nulos
print("\n📌 Valores nulos en el dataset:")
print(df.isnull().sum())

# Revisar estadísticas de columnas numéricas
print("\n📌 Resumen estadístico de columnas numéricas:")
print(df.describe())

# Revisar estadísticas de columnas categóricas
print("\n📌 Resumen estadístico de columnas categóricas:")
print(df.describe(include="object"))

,Filename,SHA256,TimeDateStamp,Subsystem,NumberOfSections,Num_DLLs,Num_Imports,Num_Suspicious_APIs,Strings
0,JH78C0A33A1B472A8C16123FD696A5CE5EBB,14af57c8a6a37b2fa9bd738285b2951b63fcfb34e1450c...,2009-05-14 17:12:40,2,4,5,62,4,4
1,NBV_8B75BCBFF174C25A0161F30758509A44,f52af02529b12c20e45618cae1a38c4ace06f5496ce160...,2009-05-14 17:12:40,2,4,5,62,4,4
2,POL55_A4F1ECC4D25B33395196B5D51A06790,57ef37c30a4a10b334a745d2f6235b4703ae99cf8e64e1...,2011-07-29 07:10:31,2,3,7,85,1,5
3,6FAA4740F99408D4D2DDDD0B09BBDEFD,a5244829aa94b009e479f06f7ae3e9f40daef32b1570c4...,2009-05-14 17:12:40,2,4,5,62,4,4
4,A316D5AECA269CA865077E7FFF356E7D,02e0d550c86910a19ab73ec02bcbe4ed095d683414db2f...,2009-05-14 17:12:40,2,4,5,62,4,4



📌 Tipos de datos en cada columna:
Filename               object
SHA256                 object
TimeDateStamp          object
Subsystem               int64
NumberOfSections        int64
Num_DLLs                int64
Num_Imports             int64
Num_Suspicious_APIs     int64
Strings                 int64
dtype: object

📌 Valores nulos en el dataset:
Filename               0
SHA256                 0
TimeDateStamp          0
Subsystem              0
NumberOfSections       0
Num_DLLs               0
Num_Imports            0
Num_Suspicious_APIs    0
Strings                0
dtype: int64

📌 Resumen estadístico de columnas numéricas:
       Subsystem  NumberOfSections   Num_DLLs  Num_Imports  \
count       40.0         40.000000  40.000000    40.000000   
mean         2.0          3.800000   5.875000   103.175000   
std          0.0          0.516398   1.522271    73.654038   
min          2.0          3.000000   4.000000    51.000000   
25%          2.0          3.750000   5.000000    62.000

In [348]:
df["Subsystem"] = df["Subsystem"].astype(int)
df["DLLCharacteristics"] = df["DLLCharacteristics"].astype(int)

if "Imports" in df.columns:
    df["Num_Imports"] = df["Imports"].apply(lambda x: len(str(x).split(",")) if pd.notnull(x) else 0)
else:
    df["Num_Imports"] = 0  # Si no existe, se asigna 0

# Revisar si la columna 'Strings' existe antes de contar las cadenas sospechosas
if "Strings" in df.columns:
    df["Num_Strings"] = df["Strings"].apply(lambda x: len(str(x).split(",")) if pd.notnull(x) else 0)
else:
    df["Num_Strings"] = 0  # Si no existe, se asigna 0


# Rellenar valores nulos antes de escalar
df["NumberOfSections"] = df["NumberOfSections"].fillna(df["NumberOfSections"].median()).astype(int)
df["Num_Imports"] = df["Num_Imports"].fillna(0)
df["Num_Strings"] = df["Num_Strings"].fillna(0)

KeyError: 'DLLCharacteristics'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 📌 Asegurar que Packed y Suspicious_Sections sean enteros (ya vienen en formato numérico)
df["Packed"] = df["Packed"].fillna(0).astype(int)
df["Suspicious_Sections"] = df["Suspicious_Sections"].fillna(0).astype(int)

# 📌 Asegurar que Subsystem y DLLCharacteristics sean enteros
df["Subsystem"] = df["Subsystem"].astype(int)
df["DLLCharacteristics"] = df["DLLCharacteristics"].astype(int)

# 📌 Contar el número de imports por archivo (solo si existe la columna)
if "Imports" in df.columns:
    df["Num_Imports"] = df["Imports"].apply(lambda x: len(str(x).split(",")) if pd.notnull(x) else 0)
else:
    df["Num_Imports"] = 0

# 📌 Contar el número de strings sospechosas por archivo (solo si existe la columna)
if "Strings" in df.columns:
    df["Num_Strings"] = df["Strings"].apply(lambda x: len(str(x).split(",")) if pd.notnull(x) else 0)
else:
    df["Num_Strings"] = 0

# 📌 Asegurar que las columnas a normalizar no tengan valores nulos
df["NumberOfSections"] = df["NumberOfSections"].fillna(df["NumberOfSections"].median()).astype(int)
df["Num_Imports"] = df["Num_Imports"].fillna(0)
df["Num_Strings"] = df["Num_Strings"].fillna(0)

# 📌 Aplicar normalización con MinMaxScaler
scaler = MinMaxScaler()
columns_to_scale = ["NumberOfSections", "Num_Imports", "Num_Strings"]
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# 📌 Guardar dataset limpio
df.to_csv("malware_dataset_cleaned.csv", index=False)
print("✅ Dataset guardado como 'malware_dataset_cleaned.csv'")

# 📌 Verificar los datos finales
display(df.head())


✅ Dataset guardado como 'malware_dataset_cleaned.csv'


,Filename,SHA256,TimeDateStamp,Subsystem,DLLCharacteristics,NumberOfSections,Packed,Suspicious_Sections,Num_DLLs,Num_Imports,Num_Suspicious_APIs,Strings,Num_Strings
0,JH78C0A33A1B472A8C16123FD696A5CE5EBB,14af57c8a6a37b2fa9bd738285b2951b63fcfb34e1450c...,2009-05-14 17:12:40,2,0,0.5,0,0,5,0.0,4,4,0.0
1,NBV_8B75BCBFF174C25A0161F30758509A44,f52af02529b12c20e45618cae1a38c4ace06f5496ce160...,2009-05-14 17:12:40,2,0,0.5,0,0,5,0.0,4,4,0.0
2,POL55_A4F1ECC4D25B33395196B5D51A06790,57ef37c30a4a10b334a745d2f6235b4703ae99cf8e64e1...,2011-07-29 07:10:31,2,0,0.0,0,0,7,0.0,1,5,0.0
3,6FAA4740F99408D4D2DDDD0B09BBDEFD,a5244829aa94b009e479f06f7ae3e9f40daef32b1570c4...,2009-05-14 17:12:40,2,0,0.5,0,0,5,0.0,4,4,0.0
4,A316D5AECA269CA865077E7FFF356E7D,02e0d550c86910a19ab73ec02bcbe4ed095d683414db2f...,2009-05-14 17:12:40,2,0,0.5,0,0,5,0.0,4,4,0.0
